# How to Load and Format Data for sktime

This guide shows you how to load data from various sources and format it correctly for use with sktime.

## Overview

This guide covers:
1. Loading data from different file formats
2. Converting to sktime-compatible formats
3. Handling different time series structures
4. Common data formatting issues and solutions

## 1. Loading Data from CSV Files

In [ ]:
import numpy as np
import pandas as pd

from sktime.datatypes import check_raise, mtype, scitype

# Create sample CSV data for demonstration
print("Creating sample data files...")

# 1. Simple time series CSV
dates = pd.date_range("2020-01-01", periods=100, freq="D")
values = np.random.randn(100).cumsum() + 100
simple_df = pd.DataFrame({"date": dates, "sales": values, "region": "North"})

print("Sample CSV structure:")
print(simple_df.head())
print(f"Shape: {simple_df.shape}")

### Loading and Converting CSV Data

In [ ]:
# Method 1: Direct conversion to time series
print("Method 1: Direct conversion")
y_simple = simple_df.set_index("date")["sales"]
print(f"Result type: {type(y_simple)}")
print(f"Scitype: {scitype(y_simple)}")
print(f"Mtype: {mtype(y_simple)}")
print(f"Index type: {type(y_simple.index)}")
print(f"Sample: {y_simple.head()}")

# Method 2: Parse dates during loading
print("\nMethod 2: Parse dates during loading")
# Simulate reading from CSV with date parsing
y_parsed = pd.Series(
    simple_df["sales"].values, index=pd.to_datetime(simple_df["date"]), name="sales"
)
print(f"Result: {y_parsed.head()}")

# Method 3: Handle different date formats
print("\nMethod 3: Different date formats")
# Create data with string dates
string_dates = ["2020-01-01", "2020-01-02", "2020-01-03", "2020-01-04", "2020-01-05"]
string_values = [100, 101, 99, 102, 98]

y_string_dates = pd.Series(
    string_values, index=pd.to_datetime(string_dates), name="values"
)
print(f"String dates converted: {y_string_dates}")

# Validate the data
try:
    check_raise(y_simple, mtype="pd.Series", scitype="Series")
    print("\n✓ Data validation successful!")
except Exception as e:
    print(f"\n✗ Data validation failed: {e}")

## 2. Loading Panel Data (Multiple Time Series)

In [ ]:
# Create panel data sample
print("Creating panel data...")

panel_data = []
regions = ["North", "South", "East", "West"]
dates = pd.date_range("2020-01-01", periods=50, freq="D")

for region in regions:
    base_value = np.random.randint(80, 120)
    trend = np.random.uniform(-0.1, 0.1)
    noise = np.random.randn(50) * 5

    values = base_value + np.arange(50) * trend + noise

    for i, (date, value) in enumerate(zip(dates, values)):
        panel_data.append(
            {
                "region": region,
                "date": date,
                "sales": value,
                "day_of_week": date.dayofweek,
            }
        )

panel_df = pd.DataFrame(panel_data)
print(f"Panel data shape: {panel_df.shape}")
print(f"Sample:\n{panel_df.head(10)}")

# Convert to sktime panel format (MultiIndex)
print("\nConverting to sktime panel format...")
panel_multiindex = panel_df.set_index(["region", "date"])["sales"]

print(f"Panel type: {type(panel_multiindex)}")
print(f"Scitype: {scitype(panel_multiindex)}")
print(f"Mtype: {mtype(panel_multiindex)}")
print(f"Shape: {panel_multiindex.shape}")
print(f"Index levels: {panel_multiindex.index.names}")
print(f"Sample:\n{panel_multiindex.head(10)}")

## 3. Handling Different Data Sources

In [ ]:
# Simulate different data source scenarios
print("Handling different data source formats:")
print("=" * 40)

# Scenario 1: Wide format data (columns are time periods)
print("\n1. Wide format data:")
wide_data = pd.DataFrame(
    {
        "series_id": ["A", "B", "C"],
        "2020-01": [100, 150, 80],
        "2020-02": [105, 145, 85],
        "2020-03": [110, 140, 90],
        "2020-04": [108, 155, 88],
    }
)
print(wide_data)

# Convert wide to long format
long_data = wide_data.melt(id_vars=["series_id"], var_name="date", value_name="value")
long_data["date"] = pd.to_datetime(long_data["date"])
panel_from_wide = long_data.set_index(["series_id", "date"])["value"]

print(f"Converted to panel format: {panel_from_wide.shape}")
print(panel_from_wide.head())

# Scenario 2: JSON-like nested data
print("\n2. JSON-like nested data:")
json_like_data = {
    "series_A": {"2020-01-01": 100, "2020-01-02": 102, "2020-01-03": 98},
    "series_B": {"2020-01-01": 150, "2020-01-02": 155, "2020-01-03": 148},
}

# Convert JSON-like to sktime format
json_data_list = []
for series_id, series_data in json_like_data.items():
    for date_str, value in series_data.items():
        json_data_list.append(
            {"series_id": series_id, "date": pd.to_datetime(date_str), "value": value}
        )

json_df = pd.DataFrame(json_data_list)
panel_from_json = json_df.set_index(["series_id", "date"])["value"]

print(f"Converted JSON data: {panel_from_json.shape}")
print(panel_from_json)

# Scenario 3: Database-like format with separate date column
print("\n3. Database format with timestamps:")
db_data = pd.DataFrame(
    {
        "timestamp": pd.date_range("2020-01-01 09:00:00", periods=24, freq="H"),
        "sensor_id": ["temp_01"] * 12 + ["temp_02"] * 12,
        "measurement": np.random.normal(20, 2, 24),
        "location": ["Room_A"] * 12 + ["Room_B"] * 12,
    }
)

print(f"Database-like data:\n{db_data.head()}")

# Convert to time series format
ts_from_db = db_data.set_index(["sensor_id", "timestamp"])["measurement"]
print(f"\nConverted to time series: {ts_from_db.shape}")
print(ts_from_db.head())

## 4. Handling Common Data Issues

In [ ]:
print("Common Data Issues and Solutions:")
print("=" * 35)

# Issue 1: Missing dates/irregular frequency
print("\n1. Missing dates and irregular frequency:")
irregular_dates = pd.to_datetime(
    [
        "2020-01-01",
        "2020-01-02",
        "2020-01-05",  # Missing 3rd and 4th
        "2020-01-06",
        "2020-01-10",  # Missing 7th, 8th, 9th
    ]
)
irregular_values = [100, 102, 105, 108, 115]
irregular_series = pd.Series(irregular_values, index=irregular_dates)

print(f"Original irregular series:\n{irregular_series}")

# Solution: Reindex to regular frequency
regular_index = pd.date_range(
    start=irregular_series.index.min(), end=irregular_series.index.max(), freq="D"
)
regular_series = irregular_series.reindex(regular_index)

print(f"\nAfter reindexing (with NaN for missing):\n{regular_series}")

# Fill missing values
filled_series = regular_series.interpolate(method="linear")
print(f"\nAfter interpolation:\n{filled_series}")

# Issue 2: Wrong data types
print("\n\n2. Wrong data types:")
wrong_types = pd.DataFrame(
    {
        "date": ["2020-01-01", "2020-01-02", "2020-01-03"],  # String dates
        "value": ["100.5", "102.3", "99.8"],  # String numbers
        "category": [1, 2, 1],  # Numeric categories
    }
)

print(f"Original data types:\n{wrong_types.dtypes}")
print(wrong_types)

# Solution: Convert data types
corrected_types = wrong_types.copy()
corrected_types["date"] = pd.to_datetime(corrected_types["date"])
corrected_types["value"] = pd.to_numeric(corrected_types["value"])
corrected_types["category"] = corrected_types["category"].astype("category")

print(f"\nCorrected data types:\n{corrected_types.dtypes}")

# Convert to time series
corrected_series = corrected_types.set_index("date")["value"]
print(f"\nFinal time series:\n{corrected_series}")

# Issue 3: Duplicate timestamps
print("\n\n3. Duplicate timestamps:")
duplicate_data = pd.DataFrame(
    {
        "date": ["2020-01-01", "2020-01-01", "2020-01-02", "2020-01-02"],
        "value": [100, 105, 102, 98],
        "source": ["A", "B", "A", "B"],
    }
)
duplicate_data["date"] = pd.to_datetime(duplicate_data["date"])

print(f"Data with duplicates:\n{duplicate_data}")

# Solution 1: Aggregate duplicates
aggregated = duplicate_data.groupby("date")["value"].mean()
print(f"\nAfter aggregation (mean):\n{aggregated}")

# Solution 2: Keep source information
multi_series = duplicate_data.set_index(["source", "date"])["value"]
print(f"\nAs panel data:\n{multi_series}")

# Issue 4: Different frequencies in same dataset
print("\n\n4. Mixed frequencies:")
daily_data = pd.DataFrame(
    {
        "date": pd.date_range("2020-01-01", periods=7, freq="D"),
        "daily_sales": np.random.randint(100, 200, 7),
    }
)

weekly_data = pd.DataFrame(
    {
        "date": pd.date_range("2020-01-01", periods=4, freq="W"),
        "weekly_budget": np.random.randint(1000, 1500, 4),
    }
)

print(f"Daily data:\n{daily_data}")
print(f"\nWeekly data:\n{weekly_data}")

# Solution: Align to common frequency
# Upsample weekly to daily
weekly_series = weekly_data.set_index("date")["weekly_budget"]
weekly_daily = weekly_series.resample("D").ffill()  # Forward fill

print(f"\nWeekly data upsampled to daily:\n{weekly_daily}")

## 5. Creating Helper Functions

In [ ]:
def load_univariate_series(data, date_col, value_col, date_format=None):
    """Load univariate time series from DataFrame.

    Parameters
    ----------
    data : pd.DataFrame or str
        DataFrame or path to CSV file
    date_col : str
        Name of date column
    value_col : str
        Name of value column
    date_format : str, optional
        Date format string

    Returns
    -------
    pd.Series
        Time series in sktime format
    """
    if isinstance(data, str):
        df = pd.read_csv(data)
    else:
        df = data.copy()

    # Convert date column
    if date_format:
        df[date_col] = pd.to_datetime(df[date_col], format=date_format)
    else:
        df[date_col] = pd.to_datetime(df[date_col])

    # Convert to time series
    series = df.set_index(date_col)[value_col]

    # Validate
    check_raise(series, mtype="pd.Series", scitype="Series")

    return series


def load_panel_data(data, instance_col, date_col, value_col, date_format=None):
    """Load panel data from DataFrame.

    Parameters
    ----------
    data : pd.DataFrame or str
        DataFrame or path to CSV file
    instance_col : str
        Name of instance/series identifier column
    date_col : str
        Name of date column
    value_col : str
        Name of value column
    date_format : str, optional
        Date format string

    Returns
    -------
    pd.Series
        Panel data in sktime format (MultiIndex)
    """
    if isinstance(data, str):
        df = pd.read_csv(data)
    else:
        df = data.copy()

    # Convert date column
    if date_format:
        df[date_col] = pd.to_datetime(df[date_col], format=date_format)
    else:
        df[date_col] = pd.to_datetime(df[date_col])

    # Convert to panel format
    panel = df.set_index([instance_col, date_col])[value_col]

    # Validate
    check_raise(panel, mtype="pd-multiindex", scitype="Panel")

    return panel


def clean_time_series(series, freq=None, fill_method="interpolate"):
    """Clean and regularize time series.

    Parameters
    ----------
    series : pd.Series
        Input time series
    freq : str, optional
        Target frequency (e.g., 'D', 'H', 'M')
    fill_method : str, default='interpolate'
        Method to fill missing values

    Returns
    -------
    pd.Series
        Cleaned time series
    """
    cleaned = series.copy()

    # Remove duplicates (keep first)
    cleaned = cleaned[~cleaned.index.duplicated(keep="first")]

    # Sort by index
    cleaned = cleaned.sort_index()

    # Regularize frequency if specified
    if freq:
        regular_index = pd.date_range(
            start=cleaned.index.min(), end=cleaned.index.max(), freq=freq
        )
        cleaned = cleaned.reindex(regular_index)

    # Fill missing values
    if fill_method == "interpolate":
        cleaned = cleaned.interpolate()
    elif fill_method == "ffill":
        cleaned = cleaned.fillna(method="ffill")
    elif fill_method == "bfill":
        cleaned = cleaned.fillna(method="bfill")

    return cleaned


# Test the helper functions
print("Testing helper functions:")
print("=" * 25)

# Test univariate loader
test_data = pd.DataFrame(
    {
        "timestamp": ["2020-01-01", "2020-01-02", "2020-01-03"],
        "sales": [100, 105, 98],
        "region": ["A", "A", "A"],
    }
)

series_result = load_univariate_series(test_data, "timestamp", "sales")
print("\nUnivariate series loaded:")
print(series_result)

# Test panel loader
panel_test_data = pd.DataFrame(
    {
        "series_id": ["A", "A", "B", "B"],
        "date": ["2020-01-01", "2020-01-02", "2020-01-01", "2020-01-02"],
        "value": [100, 105, 90, 92],
    }
)

panel_result = load_panel_data(panel_test_data, "series_id", "date", "value")
print("\nPanel data loaded:")
print(panel_result)

# Test cleaning function
messy_series = pd.Series(
    [100, np.nan, 105, 110],
    index=pd.to_datetime(["2020-01-01", "2020-01-03", "2020-01-04", "2020-01-06"]),
)

cleaned_result = clean_time_series(messy_series, freq="D")
print("\nCleaned series:")
print(cleaned_result)

print("\n✓ All helper functions working correctly!")

## 6. Real-World Examples

In [ ]:
print("Real-World Data Loading Examples:")
print("=" * 34)

# Example 1: Stock price data
print("\n1. Stock Price Data Format:")
stock_data = pd.DataFrame(
    {
        "Date": pd.date_range("2020-01-01", periods=10, freq="B"),  # Business days
        "Open": np.random.uniform(100, 110, 10),
        "High": np.random.uniform(110, 120, 10),
        "Low": np.random.uniform(90, 100, 10),
        "Close": np.random.uniform(95, 115, 10),
        "Volume": np.random.randint(1000000, 5000000, 10),
    }
)

print(stock_data.head())

# Convert to time series for closing prices
close_prices = stock_data.set_index("Date")["Close"]
print(f"\nClose prices as time series: {scitype(close_prices)}")

# Example 2: IoT sensor data
print("\n\n2. IoT Sensor Data Format:")
iot_data = pd.DataFrame(
    {
        "timestamp": pd.date_range("2020-01-01 00:00:00", periods=48, freq="30min"),
        "device_id": ["sensor_01"] * 24 + ["sensor_02"] * 24,
        "temperature": np.random.normal(22, 3, 48),
        "humidity": np.random.normal(60, 10, 48),
        "battery_level": np.random.uniform(20, 100, 48),
    }
)

print(iot_data.head())

# Convert to panel data for temperature readings
temp_panel = iot_data.set_index(["device_id", "timestamp"])["temperature"]
print(f"\nTemperature panel data: {scitype(temp_panel)}")
print(f"Shape: {temp_panel.shape}")

# Example 3: Sales data with hierarchy
print("\n\n3. Hierarchical Sales Data:")
sales_data = pd.DataFrame(
    {
        "date": pd.date_range("2020-01-01", periods=12, freq="M"),
        "country": ["USA"] * 6 + ["Canada"] * 6,
        "state": ["CA", "NY", "TX"] * 2 + ["ON", "BC"] * 3,
        "product": ["A", "B"] * 6,
        "sales": np.random.randint(1000, 5000, 12),
    }
)

print(sales_data.head())

# Different grouping strategies
# By country
country_sales = sales_data.groupby(["country", "date"])["sales"].sum().reset_index()
country_panel = country_sales.set_index(["country", "date"])["sales"]
print(f"\nCountry-level panel: {country_panel.shape}")

# By product
product_sales = sales_data.groupby(["product", "date"])["sales"].sum().reset_index()
product_panel = product_sales.set_index(["product", "date"])["sales"]
print(f"Product-level panel: {product_panel.shape}")

# Example 4: Web analytics data
print("\n\n4. Web Analytics Data:")
web_data = pd.DataFrame(
    {
        "date": pd.date_range("2020-01-01", periods=30, freq="D"),
        "page_views": np.random.poisson(1000, 30),
        "unique_visitors": np.random.poisson(500, 30),
        "bounce_rate": np.random.uniform(0.3, 0.7, 30),
        "conversion_rate": np.random.uniform(0.01, 0.05, 30),
    }
)

print(web_data.head())

# Convert multiple metrics to separate series
page_views_ts = web_data.set_index("date")["page_views"]
conversion_ts = web_data.set_index("date")["conversion_rate"]

print(f"\nPage views time series: {page_views_ts.shape}")
print(f"Conversion rate time series: {conversion_ts.shape}")

# Combine into multivariate series if needed
multivariate_df = web_data.set_index("date")[["page_views", "conversion_rate"]]
print(f"\nMultivariate data: {multivariate_df.shape}")
print(f"Scitype: {scitype(multivariate_df)}")

## 7. Validation and Quality Checks

In [ ]:
def validate_time_series_data(data, name="data"):
    """Comprehensive validation of time series data.

    Parameters
    ----------
    data : pd.Series or pd.DataFrame
        Time series data to validate
    name : str
        Name for reporting

    Returns
    -------
    dict
        Validation report
    """
    report = {"name": name, "issues": [], "warnings": [], "info": []}

    # Basic type checks
    if not isinstance(data, (pd.Series, pd.DataFrame)):
        report["issues"].append(f"Data is not pandas Series or DataFrame: {type(data)}")
        return report

    # Index checks
    if not isinstance(data.index, (pd.DatetimeIndex, pd.MultiIndex)):
        report["issues"].append(
            f"Index is not DatetimeIndex or MultiIndex: {type(data.index)}"
        )

    # Check for missing values
    if (
        data.isnull().any().any()
        if isinstance(data, pd.DataFrame)
        else data.isnull().any()
    ):
        missing_count = (
            data.isnull().sum().sum()
            if isinstance(data, pd.DataFrame)
            else data.isnull().sum()
        )
        report["warnings"].append(f"Contains {missing_count} missing values")

    # Check for duplicates in index
    if data.index.duplicated().any():
        dup_count = data.index.duplicated().sum()
        report["warnings"].append(f"Contains {dup_count} duplicate index values")

    # Check data types
    if isinstance(data, pd.Series):
        if not pd.api.types.is_numeric_dtype(data):
            report["warnings"].append(f"Series is not numeric: {data.dtype}")
    else:
        non_numeric = [
            col for col in data.columns if not pd.api.types.is_numeric_dtype(data[col])
        ]
        if non_numeric:
            report["warnings"].append(f"Non-numeric columns: {non_numeric}")

    # Check frequency regularity (for DatetimeIndex)
    if isinstance(data.index, pd.DatetimeIndex):
        try:
            freq = pd.infer_freq(data.index)
            if freq is None:
                report["warnings"].append("Could not infer regular frequency")
            else:
                report["info"].append(f"Inferred frequency: {freq}")
        except Exception:
            report["warnings"].append("Error inferring frequency")

    # Size checks
    report["info"].append(f"Shape: {data.shape}")
    report["info"].append(f"Date range: {data.index.min()} to {data.index.max()}")

    # sktime compatibility
    try:
        check_raise(data, mtype=mtype(data), scitype=scitype(data))
        report["info"].append(f"✓ sktime compatible: {scitype(data)}/{mtype(data)}")
    except Exception as e:
        report["issues"].append(f"Not sktime compatible: {str(e)[:100]}")

    return report


def print_validation_report(report):
    """Print formatted validation report."""
    print(f"\nValidation Report for '{report['name']}':")
    print("=" * (len(report["name"]) + 25))

    if report["issues"]:
        print("\n❌ ISSUES:")
        for issue in report["issues"]:
            print(f"   • {issue}")

    if report["warnings"]:
        print("\n⚠️  WARNINGS:")
        for warning in report["warnings"]:
            print(f"   • {warning}")

    if report["info"]:
        print("\nℹ️  INFO:")
        for info in report["info"]:
            print(f"   • {info}")

    # Overall status
    if not report["issues"]:
        if not report["warnings"]:
            print("\n✅ Status: EXCELLENT - No issues or warnings")
        else:
            print("\n✅ Status: GOOD - No critical issues, some warnings")
    else:
        print("\n❌ Status: NEEDS ATTENTION - Critical issues found")


# Test validation function
print("Testing data validation:")

# Good data
good_data = pd.Series(
    np.random.randn(30),
    index=pd.date_range("2020-01-01", periods=30, freq="D"),
    name="good_series",
)

good_report = validate_time_series_data(good_data, "Good Time Series")
print_validation_report(good_report)

# Problematic data
bad_data = pd.Series(
    [100, np.nan, "invalid", 105, 102],
    index=[1, 2, 2, 3, 4],  # Non-datetime index with duplicates
    name="bad_series",
)

bad_report = validate_time_series_data(bad_data, "Problematic Data")
print_validation_report(bad_report)

## Summary

This guide covered comprehensive data loading and formatting for sktime:

### Key Topics Covered:

1. **CSV Data Loading**: Converting flat files to time series format
2. **Panel Data**: Handling multiple time series with MultiIndex
3. **Different Sources**: Wide format, JSON-like, database formats
4. **Common Issues**: Missing dates, wrong types, duplicates, mixed frequencies
5. **Helper Functions**: Reusable utilities for data loading
6. **Real-World Examples**: Stock, IoT, sales, web analytics data
7. **Validation**: Comprehensive data quality checking

### Best Practices:

- **Always validate** data after loading
- **Use DatetimeIndex** for time series data
- **Handle missing values** appropriately
- **Check data types** and convert as needed
- **Use MultiIndex** for panel data
- **Create reusable functions** for common patterns
- **Test with sktime compatibility** functions

### Next Steps:

- Apply these techniques to your own data
- Explore "Handle Missing Values" guide for advanced missing data techniques
- Learn "Convert Between Data Formats" for more transformation options
- Try forecasting tutorials with your formatted data